# Performance Report

This notebooks provides an overview for using and understanding the performance report check.

**Structure:**

- [What is the purpose of the check?](#purpose)
- [Generate data & model](#generate_data_model)
- [Run the check](#run_check)
- [Define a condition](#define_condition)

## What is the purpose of the check? <a name='purpose'></a>

The performance report is designed to provide valuable information about the model performance. This checks evaluates
the given model on the given data and returns informative results on various metrics. By default, the metrics that are being evaluated are common metrics that are being used to evaluate models of the same task type. For example - for classfication the default metrics will be the precision & recall per class, and for object detection it will be the Mean Average Precision per class.

In addition, the checks supports any custom metric provided by the user, as long as this metrics implement the API of [torch.ignite.Metric](https://pytorch.org/ignite/metrics.html#how-to-create-a-custom-metric). In order to use custom metrics, a list of the custom metrics should be passed to the `alternative_metrics` parameter of the check.

### Imports

In [1]:
from deepchecks.vision.base import VisionData

In [2]:
from deepchecks.vision.checks.performance import PerformanceReport

## Classification Performance Report


### Generate data and model:

In [3]:
from deepchecks.vision.datasets.classification import mnist

In [4]:
mnist_model = mnist.load_model()

In [5]:
train_ds = mnist.load_dataset(train=True, object_type='Dataset')
test_ds = mnist.load_dataset(train=False, object_type='Dataset')

### Defining the prediction formatter

The model returns the logic of the neural network, and we need to convert it to the format accepted by deepchecks, which is an array of shape (N, n_classes). N is the batch size, and each element is an array of length n_classes that represent the probability of the each class. 

In order to achieve this, we can simply apply a softmax function on the outputs and pass it to the ClassificationPredictionFormatter:

In [10]:
from deepchecks.vision.utils.classification_formatters import ClassificationPredictionFormatter
from torch import nn

pred_formatter = ClassificationPredictionFormatter(nn.Softmax(dim=1))

In [11]:
from deepchecks.vision.utils.classification_formatters import ClassificationPredictionFormatter
from torch import nn
check = PerformanceReport(prediction_formatter=pred_formatter)
check.run(train_ds, test_ds, mnist_model)

/Users/itaygabbay/Deepchecks/deepchecks/deepchecks/vision/datasets/classification/mnist.py:201: UserWarning:

Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Performance Report Summarize given metrics on a dataset and model. Read More... Additional Outputs

## Object Detection Performance Report

For object detection tasks - the default metric that is being calculated it the Mean Average Precision. The definition of the Mean Average Precision is identical to how the COCO dataset defined it - mean of the average precision per class, over the range [0.5, 0.95, 0.05] of IoU thresholds.

In [12]:
import numpy as np
from deepchecks.vision.utils.detection_formatters import DetectionLabelFormatter, DetectionPredictionFormatter
from deepchecks.vision.datasets.detection import coco


### Generate Data and Model

We generate a sample dataset of 128 images from the [COCO dataset](https://cocodataset.org/#home), and using the [YOLOv5 model](https://github.com/ultralytics/yolov5).

For the label formatter - our dataset returns exactly the accepted format, so our formatting function is the simple `lambda x: x` function.

In [14]:
yolo = coco.load_model(pretrained=True)

coco_train_loader = coco.load_dataset(train=True)
coco_test_loader = coco.load_dataset(train=False)

train_ds = VisionData(coco_train_loader, label_transformer=DetectionLabelFormatter(lambda x: x), num_classes=80)
test_ds = VisionData(coco_test_loader, label_transformer=DetectionLabelFormatter(lambda x: x), num_classes=80)

Using cache found in /Users/itaygabbay/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-1-18 torch 1.10.1 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Using downloaded and verified file: /Users/itaygabbay/Deepchecks/deepchecks/deepchecks/vision/datasets/detection/coco128.zip
Extracting /Users/itaygabbay/Deepchecks/deepchecks/deepchecks/vision/datasets/detection/coco128.zip to /Users/itaygabbay/Deepchecks/deepchecks/deepchecks/vision/datasets/detection
Using downloaded and verified file: /Users/itaygabbay/Deepchecks/deepchecks/deepchecks/vision/datasets/detection/coco128.zip
Extracting /Users/itaygabbay/Deepchecks/deepchecks/deepchecks/vision/datasets/detection/coco128.zip to /Users/itaygabbay/Deepchecks/deepchecks/deepchecks/vision/datasets/detection


### Defining the prediction formatter

The COCO module implemented in deepchecks alreaady contains a transformation function to convert the yolo detection to the correct format of [x, y, w, h, confidence, class_id].

In [15]:
from deepchecks.vision.utils.detection_formatters import DetectionPredictionFormatter

det_formatter = DetectionPredictionFormatter(coco.yolo_wrapper)

In [17]:
check = PerformanceReport(prediction_formatter=det_formatter)
check.run(train_ds, test_ds, yolo)

/Users/itaygabbay/.virtualenvs/mlchecks/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/Users/itaygabbay/.virtualenvs/mlchecks/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/Users/itaygabbay/.virtualenvs/mlchecks/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/Users/itaygabbay/.virtualenvs/mlchecks/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling

/Users/itaygabbay/.virtualenvs/mlchecks/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning:

User provided device_type of 'cuda', but CUDA is not available. Disabling



Performance Report Summarize given metrics on a dataset and model. Read More... Additional Outputs